In [4]:
from project.rebel import make_kb

text = 'Punta Cana is a resort town in the municipality of Higüey, in La Altagracia Province, the easternmost province of the Dominican Republic.'

kb = make_kb(text)

In [2]:
import pandas as pd

pd.DataFrame(kb, columns=['head', 'relation', 'tail'])

,head,relation,tail
0,Higüey,country,Dominican Republic
1,Punta Cana,country,Dominican Republic
2,province,country,Dominican Republic
3,La Altagracia Province,instance of,province
4,La Altagracia Province,country,Dominican Republic
5,Dominican Republic,contains administrative territorial entity,La Altagracia Province
6,Higüey,located in the administrative territorial entity,La Altagracia Province
7,Punta Cana,located in the administrative territorial entity,La Altagracia Province
